# Azure AI Agent service - Many agents

<img src="agent.jpg" width=800>
<img src="foundry.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import os
import sys

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
from openai import AzureOpenAI

In [2]:
load_dotenv("azure.env")

True

In [3]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

## Project

In [4]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.getenv("PROJECT_CONNECTION_STRING"),
)

In [5]:
model = "gpt-4o"

In [6]:
agent1 = project_client.agents.create_agent(
    model=model,
    name="Q&A",
    instructions=("You are an AI agent."),
)

print(f"Created Q&A agent, agent ID: {agent1.id}")

Created Q&A agent, agent ID: asst_5yfHq0imn6qhanM3oqg9V8c0


In [7]:
agent2 = project_client.agents.create_agent(
    model=model,
    name="Translator",
    instructions=("You are an AI agent that translate english into french")
)

print(f"Created Translator agent, agent ID: {agent2.id}")

Created Translator agent, agent ID: asst_bN84KgECvNaP3CP2dT4a49Zl


In [8]:
agent3 = project_client.agents.create_agent(
    model=model,
    name="Emojis",
    instructions=("You are an AI agent that add emojis to a text")
)

print(f"Created Emojis agent, agent ID: {agent3.id}")

Created Emojis agent, agent ID: asst_ZcAgUnlru1tM4IFt3ONmosm0


In [9]:
thread = project_client.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

Created thread, thread ID: thread_vPB9YMfkVYt191vvWRDV8kmD


In [10]:
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=("What is a GenAI model?"),
)

print(f"Message created, message ID: {message.id}")

Message created, message ID: msg_BJVfklTjAct7jLFZofzEexKE


In [11]:
run = project_client.agents.create_and_process_run(thread_id=thread.id,
                                                   agent_id=agent1.id)

print(f"Run finished for Agent1 with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished for Agent1 with status: completed


In [12]:
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Thread messages: {messages}")

# Print last message
last_msg = messages.get_last_text_message_by_role("assistant")

if last_msg:
    print(f"\nAgent1 last message:\n{last_msg.text.value}")

Thread messages: {'object': 'list', 'data': [{'id': 'msg_QFJZWx87rfQlVfkSitRX2OFX', 'object': 'thread.message', 'created_at': 1745961651, 'assistant_id': 'asst_5yfHq0imn6qhanM3oqg9V8c0', 'thread_id': 'thread_vPB9YMfkVYt191vvWRDV8kmD', 'run_id': 'run_tDk74kTFx2xpLlexk0nk0ykT', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'A **GenAI model**, short for **Generative Artificial Intelligence model**, is an AI system designed to generate new, original outputs based on patterns and knowledge learned from existing data. Unlike traditional AI systems that primarily classify or predict existing data, GenAI models can create text, images, audio, videos, code, and other types of content, making them versatile and creative tools.\n\n### Key Features of GenAI Models:\n1. **Generative Nature**:\n   - They produce new content, such as text, designs, or images, rather than solely analyzing existing data.\n   \n2. **Training on Large Datasets**:\n   - These models are trained on ma

In [13]:
run = project_client.agents.create_and_process_run(thread_id=thread.id,
                                                   agent_id=agent2.id)

print(f"Run finished for Agent2 with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished for Agent2 with status: completed


In [14]:
messages = project_client.agents.list_messages(thread_id=thread.id)

# Print last message
last_msg = messages.get_last_text_message_by_role("assistant")

if last_msg:
    print(f"Agent last message:\n{last_msg.text.value}")

Agent last message:
A GenAI model, en français, est un **modèle d'Intelligence Artificielle Générative** conçu pour créer du contenu original en se basant sur des données existantes. Contrairement aux modèles classiques qui analysent ou classifient les données, les modèles GenAI peuvent générer du texte, des images, du son, des vidéos, du code et d'autres types de contenu.

### Caractéristiques principales des modèles GenAI :
1. **Capacité générative** : 
   - Ils produisent des contenus nouveaux (comme des textes, des designs ou des images) au lieu de se limiter à interpréter des données.

2. **Apprentissage sur de vastes ensembles de données** :
   - Les modèles sont entraînés sur des bases de données massives contenant divers exemples du type de contenu qu'ils doivent créer.

3. **Modèles de base ("Foundation Models")** :
   - Les systèmes GenAI reposent sur des modèles de base (par exemple, GPT d'OpenAI, Stable Diffusion pour les images) et peuvent être ajustés pour effectuer des t

In [15]:
run = project_client.agents.create_and_process_run(thread_id=thread.id,
                                                   agent_id=agent3.id)

print(f"Run finished for Agent3 with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished for Agent3 with status: completed


In [16]:
messages = project_client.agents.list_messages(thread_id=thread.id)

# Print last message
last_msg = messages.get_last_text_message_by_role("assistant")

if last_msg:
    print(f"Agent last message:\n{last_msg.text.value}")

Agent last message:
Oh, pardon ! 🎉 Je vais rendre ça encore plus fun avec des emojis 🧠✨.

---

Un **modèle GenAI** (🎨🤖) est un type d'Intelligence Artificielle Générative capable de créer du contenu original (textes 📝, images 🖼, code 💻, audio 🎵, vidéos 📹, etc.) à partir de données existantes. Contrairement aux modèles d'IA traditionnels 🔍📊, qui se concentrent sur l'analyse ou la classification, les GenAI produisent des résultats nouveaux et créatifs 🌟.

---

### 🧾 **Caractéristiques des modèles GenAI :**  
1. **Capacité générative** 🧠 : Ces modèles génèrent des contenus complètement nouveaux, comme des poèmes, des designs ou même des romans.  
2. **Entraînements sur des BIG datasets 💾📡** : Ils apprennent grâce à des millions de données textuelles, visuelles ou audio pour comprendre les patterns et les contextes.  
3. **Modèles de base ("Foundation Models")** ⚙️ : Ils servent de base pour ajuster ces modèles à des industries spécifiques, comme la santé 🏥, la finance 💰, ou le divertissem

## Post processing

In [17]:
agents = project_client.agents.list_agents()

for i in range(len(agents.data)):
    print(agents.data[i])
    print()

{'id': 'asst_ZcAgUnlru1tM4IFt3ONmosm0', 'object': 'assistant', 'created_at': 1745961648, 'name': 'Emojis', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are an AI agent that add emojis to a text', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}

{'id': 'asst_bN84KgECvNaP3CP2dT4a49Zl', 'object': 'assistant', 'created_at': 1745961647, 'name': 'Translator', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are an AI agent that translate english into french', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}

{'id': 'asst_5yfHq0imn6qhanM3oqg9V8c0', 'object': 'assistant', 'created_at': 1745961647, 'name': 'Q&A', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are an AI agent.', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}



In [18]:
project_client.agents.delete_agent(agent1.id)

{'id': 'asst_5yfHq0imn6qhanM3oqg9V8c0', 'object': 'assistant.deleted', 'deleted': True}

In [19]:
project_client.agents.delete_agent(agent2.id)

{'id': 'asst_bN84KgECvNaP3CP2dT4a49Zl', 'object': 'assistant.deleted', 'deleted': True}

In [20]:
project_client.agents.delete_agent(agent3.id)

{'id': 'asst_ZcAgUnlru1tM4IFt3ONmosm0', 'object': 'assistant.deleted', 'deleted': True}